In [ ]:
library(BiocManager)
library(BSgenome.Hsapiens.UCSC.hg38)
library(ArchR)
library(ggplot2)
library(TFBSTools)
library(Seurat)
library(ggplot2)
library(dplyr)
library(harmony)
library(SeuratData)
library(Signac)
library(BSgenome.Hsapiens.UCSC.hg38)
library(JASPAR2018)

data("geneAnnoHg38")
data("genomeAnnoHg38")
geneAnno <- geneAnnoHg38
genomeAnno <- genomeAnnoHg38
addArchRThreads(28)


In [ ]:
fn <- unclass(lsf.str(envir = asNamespace("ArchR"), all = TRUE))
  for(i in seq_along(fn)){
    tryCatch({
      eval(parse(text=paste0(fn[i], '<-ArchR:::', fn[i])))
    }, error = function(x){
    })
  }

In [ ]:
#Load the differentiation data project with combined peakSet
proj<-loadArchRProject('path to invitro project with combined peakset')
#
proj

In [ ]:
#First we need to subset both the fetal and invitro imputer RNA object to have the same gene
se1<-getMatrixFromProject(proj,useMatrix = "GeneIntegrationMatrix1")
dim(se1)
se1

In [ ]:
#Loading the invitro cleaned seurat for the friedman data
gene_integration_matrix1<-readRDS(' path to seurat friedman et al data')
DimPlot(gene_integration_matrix1, group.by = "seurat_clusters",label=TRUE)
gene_integration_matrix1


In [ ]:
#Loading the fetal heart cleaned seurat for the scRNA data
invivo_matrix<-readRDS('path to cleaned seurat data for the fetal heart scRNA')
invivo_matrix


In [ ]:
#Getting the gene names in both the object
fetal_genenames<-rownames(invivo_matrix[['RNA']]@meta.features)
invitro_genenames<-rownames(gene_integration_matrix1[['RNA']]@meta.features)

In [ ]:
#subsetting the objects with overlapping genes
fetal_subsetted<-fetal_genenames[fetal_genenames %in% invitro_genenames ]
subset1<-fetal_subsetted[fetal_subsetted %in% rowData(se1)$name ]
subset2<-invitro_genenames[invitro_genenames %in% rowData(se1)$name ]
required_genes<-subset2[subset2 %in% subset1]
length(c)



In [ ]:
#subsetting the friedman data for required genes
gene_integration_matrix<-subset(x = gene_integration_matrix1,features=c)


In [ ]:
#redoing the gene intergration for the invitro object with the combined peakset with subsetted genes
proj <- addGeneIntegrationMatrix(
    ArchRProj = proj, 
    useMatrix = "GeneScoreMatrix",
    matrixName = "GeneIntegrationMatrix1",
    reducedDims = "IterativeLSI",
    seRNA = gene_integration_matrix,
    addToArrow = TRUE,
    sampleCellsATAC = 70000,
    sampleCellsRNA = 70000,
    groupATAC = "Clusters1",
    groupRNA  = "seurat_clusters",
    nameCell  = "predictedCell_Un1",
    nameGroup = "predictedGroup_Un1",
    nameScore = "predictedScore_Un1",
    force = TRUE
)

In [ ]:
#making sure the peakset is the combined peakset
#cardiomyogenesis/combined_peak_set.csv
combinned_peakset<-read.csv(' path to the combined peakset')
head(combinned_peakset)
dim(combinned_peakset)
proj<-addPeakSet(proj,GRanges(combinned_peakset),force=TRUE)
proj<-addPeakMatrix(proj,force=TRUE)

In [ ]:
#Modifying the ArchR function to make use of specific cells as we want to look only at the end cell types.
addPeak2GeneLinks<-function (ArchRProj = NULL, reducedDims = "IterativeLSI", useMatrix = "GeneIntegrationMatrix", 
    dimsToUse = 1:30, scaleDims = NULL, cellsToUse = NULL, corCutOff = 0.75, k = 100, 
    knnIteration = 500, overlapCutoff = 0.8, maxDist = 250000, 
    scaleTo = 10^4, log2Norm = TRUE, predictionCutoff = 0.4, 
    seed = 1, threads = max(floor(getArchRThreads()/2), 1), verbose = TRUE, 
    logFile = createLogFile("addPeak2GeneLinks")) 
{
    .validInput(input = ArchRProj, name = "ArchRProj", valid = c("ArchRProj"))
    .validInput(input = reducedDims, name = "reducedDims", valid = c("character"))
    .validInput(input = dimsToUse, name = "dimsToUse", valid = c("numeric", 
        "null"))
    .validInput(input = cellsToUse, name = "cellsToUse", valid = c("character", 
        "null"))
    .validInput(input = scaleDims, name = "scaleDims", valid = c("boolean", 
        "null"))
    .validInput(input = corCutOff, name = "corCutOff", valid = c("numeric", 
        "null"))
    .validInput(input = k, name = "k", valid = c("integer"))
    .validInput(input = knnIteration, name = "knnIteration", 
        valid = c("integer"))
    .validInput(input = overlapCutoff, name = "overlapCutoff", 
        valid = c("numeric"))
    
    .validInput(input = maxDist, name = "maxDist", valid = c("integer"))
    .validInput(input = scaleTo, name = "scaleTo", valid = c("numeric"))
    .validInput(input = log2Norm, name = "log2Norm", valid = c("boolean"))
    .validInput(input = threads, name = "threads", valid = c("integer"))
    .validInput(input = verbose, name = "verbose", valid = c("boolean"))
    .validInput(input = logFile, name = "logFile", valid = c("character"))
    tstart <- Sys.time()
    .startLogging(logFile = logFile)
    .logThis(mget(names(formals()), sys.frame(sys.nframe())), 
        "addPeak2GeneLinks Input-Parameters", logFile = logFile)
    .logDiffTime(main = "Getting Available Matrices", t1 = tstart, 
        verbose = verbose, logFile = logFile)
    AvailableMatrices <- getAvailableMatrices(ArchRProj)
    if ("PeakMatrix" %ni% AvailableMatrices) {
        stop("PeakMatrix not in AvailableMatrices")
    }
    if (useMatrix %ni% AvailableMatrices) {
        stop(paste0(useMatrix, " not in AvailableMatrices"))
    }
    ArrowFiles <- getArrowFiles(ArchRProj)
    tstart <- Sys.time()
    dfAll <- .safelapply(seq_along(ArrowFiles), function(x) {
        DataFrame(cellNames = paste0(names(ArrowFiles)[x], "#", 
            h5read(ArrowFiles[x], paste0(useMatrix, "/Info/CellNames"))), 
            predictionScore = h5read(ArrowFiles[x], paste0(useMatrix, 
                "/Info/predictionScore")))
    }, threads = threads) %>% Reduce("rbind", .)
    .logDiffTime(sprintf("Filtered Low Prediction Score Cells (%s of %s, %s)", 
        sum(dfAll[, 2] < predictionCutoff), nrow(dfAll), round(sum(dfAll[, 
            2] < predictionCutoff)/nrow(dfAll), 3)), t1 = tstart, 
        verbose = verbose, logFile = logFile)
    keep <- sum(dfAll[, 2] >= predictionCutoff)/nrow(dfAll)
    dfAll <- dfAll[which(dfAll[, 2] > predictionCutoff), ]
    set.seed(seed)
    
    peakSet <- getPeakSet(ArchRProj)
    #add the code here for looking at peaks that have links only in invivo
    #
    .logThis(peakSet, "peakSet", logFile = logFile)
    geneSet <- .getFeatureDF(ArrowFiles, useMatrix, threads = threads)
    geneStart <- GRanges(geneSet$seqnames, IRanges(geneSet$start, 
        width = 1), name = geneSet$name, idx = geneSet$idx)
    .logThis(geneStart, "geneStart", logFile = logFile)
    rD <- getReducedDims(ArchRProj, reducedDims = reducedDims, 
        corCutOff = corCutOff, dimsToUse = dimsToUse)
    if (!is.null(cellsToUse)) {
        rD <- rD[cellsToUse, , drop = FALSE]
    }
    idx <- sample(seq_len(nrow(rD)), knnIteration, replace = !nrow(rD) >= 
        knnIteration)
    .logDiffTime(main = "Computing KNN", t1 = tstart, verbose = verbose, 
        logFile = logFile)
    knnObj <- .computeKNN(data = rD, query = rD[idx, ], k = k)
    .logDiffTime(main = "Identifying Non-Overlapping KNN pairs", 
        t1 = tstart, verbose = verbose, logFile = logFile)
    keepKnn <- determineOverlapCpp(knnObj, floor(overlapCutoff * 
        k))
    knnObj <- knnObj[keepKnn == 0, ]
    .logDiffTime(paste0("Identified ", nrow(knnObj), " Groupings!"), 
        t1 = tstart, verbose = verbose, logFile = logFile)
    knnObj <- lapply(seq_len(nrow(knnObj)), function(x) {
        rownames(rD)[knnObj[x, ]]
    }) %>% SimpleList
    chri <- gtools::mixedsort(unique(paste0(seqnames(peakSet))))
    chrj <- gtools::mixedsort(unique(paste0(seqnames(geneStart))))
    chrij <- intersect(chri, chrj)
    geneDF <- mcols(geneStart)
    peakDF <- mcols(peakSet)
    geneDF$seqnames <- seqnames(geneStart)
    peakDF$seqnames <- seqnames(peakSet)
    .logDiffTime(main = "Getting Group RNA Matrix", t1 = tstart, 
        verbose = verbose, logFile = logFile)
    groupMatRNA <- .getGroupMatrix(ArrowFiles = getArrowFiles(ArchRProj), 
        featureDF = geneDF, groupList = knnObj, useMatrix = useMatrix, 
        threads = threads, verbose = FALSE)
    .logThis(groupMatRNA, "groupMatRNA", logFile = logFile)
    .logDiffTime(main = "Getting Group ATAC Matrix", t1 = tstart, 
        verbose = verbose, logFile = logFile)
    groupMatATAC <- .getGroupMatrix(ArrowFiles = getArrowFiles(ArchRProj), 
        featureDF = peakDF, groupList = knnObj, useMatrix = "PeakMatrix", 
        threads = threads, verbose = FALSE)
    .logThis(groupMatATAC, "groupMatATAC", logFile = logFile)
    .logDiffTime(main = "Normalizing Group Matrices", t1 = tstart, 
        verbose = verbose, logFile = logFile)
    groupMatRNA <- t(t(groupMatRNA)/colSums(groupMatRNA)) * scaleTo
    groupMatATAC <- t(t(groupMatATAC)/colSums(groupMatATAC)) * 
        scaleTo
    if (log2Norm) {
        groupMatRNA <- log2(groupMatRNA + 1)
        groupMatATAC <- log2(groupMatATAC + 1)
    }
    names(geneStart) <- NULL
    seRNA <- SummarizedExperiment(assays = SimpleList(RNA = groupMatRNA), 
        rowRanges = geneStart)
    metadata(seRNA)$KNNList <- knnObj
    .logThis(seRNA, "seRNA", logFile = logFile)
    names(peakSet) <- NULL
    seATAC <- SummarizedExperiment(assays = SimpleList(ATAC = groupMatATAC), 
        rowRanges = peakSet)
    metadata(seATAC)$KNNList <- knnObj
    .logThis(seATAC, "seATAC", logFile = logFile)
    rm(groupMatRNA, groupMatATAC)
    gc()
    .logDiffTime(main = "Finding Peak Gene Pairings", t1 = tstart, 
        verbose = verbose, logFile = logFile)
    o <- DataFrame(findOverlaps(.suppressAll(resize(seRNA, 2 * 
        maxDist + 1, "center")), resize(rowRanges(seATAC), 1, 
        "center"), ignore.strand = TRUE))
    o$distance <- distance(rowRanges(seRNA)[o[, 1]], rowRanges(seATAC)[o[, 
        2]])
    colnames(o) <- c("B", "A", "distance")
    .logDiffTime(main = "Computing Correlations", t1 = tstart, 
        verbose = verbose, logFile = logFile)
    o$Correlation <- rowCorCpp(as.integer(o$A), as.integer(o$B), 
        assay(seATAC), assay(seRNA))
    o$VarAssayA <- .getQuantiles(matrixStats::rowVars(assay(seATAC)))[o$A]
    o$VarAssayB <- .getQuantiles(matrixStats::rowVars(assay(seRNA)))[o$B]
    o$TStat <- (o$Correlation/sqrt((1 - o$Correlation^2)/(ncol(seATAC) - 
        2)))
    o$Pval <- 2 * pt(-abs(o$TStat), ncol(seATAC) - 2)
    o$FDR <- p.adjust(o$Pval, method = "fdr")
    out <- o[, c("A", "B", "Correlation", "FDR", "VarAssayA", 
        "VarAssayB")]
    colnames(out) <- c("idxATAC", "idxRNA", "Correlation", "FDR", 
        "VarQATAC", "VarQRNA")
    mcols(peakSet) <- NULL
    names(peakSet) <- NULL
    metadata(out)$peakSet <- peakSet
    metadata(out)$geneSet <- geneStart
    dir.create(file.path(getOutputDirectory(ArchRProj), "Peak2GeneLinks"), 
        showWarnings = FALSE)
    outATAC <- file.path(getOutputDirectory(ArchRProj), "Peak2GeneLinks", 
        "seATAC-Group-KNN.rds")
    saveRDS(seATAC, outATAC, compress = FALSE)
    outRNA <- file.path(getOutputDirectory(ArchRProj), "Peak2GeneLinks", 
        "seRNA-Group-KNN.rds")
    saveRDS(seRNA, outRNA, compress = FALSE)
    metadata(out)$seATAC <- outATAC
    metadata(out)$seRNA <- outRNA
    metadata(ArchRProj@peakSet)$Peak2GeneLinks <- out
    .logDiffTime(main = "Completed Peak2Gene Correlations!", 
        t1 = tstart, verbose = verbose, logFile = logFile)
    .endLogging(logFile = logFile)
    ArchRProj
}

In [ ]:
t_coldata<-getCellColData(proj)

cells_for_peak2gene<-rownames(t_coldata[t_coldata$Clusters1 %in% c('H_Mature Cardiomyocytes','G_Primitive Cardiomyocytes' ),])



In [ ]:
proj <- addPeak2GeneLinks(
    ArchRProj = proj,
    reducedDims = "IterativeLSI",
    useMatrix = "GeneIntegrationMatrix1",
    cellsToUse=cells_for_peak2gene
)


In [ ]:
#looking at only invivo links using peaks on invitro
p2g <- metadata(proj@peakSet)$Peak2GeneLinks
p2g <- p2g[which(p2g$Correlation >= 0.45 & p2g$FDR <= 1e-04), , drop = FALSE]
head(p2g)
mATAC <- readRDS(metadata(p2g)$seATAC)[p2g$idxATAC, ]
mRNA <- readRDS(metadata(p2g)$seRNA)[p2g$idxRNA, ]
p2g$peak <- paste0(rowRanges(mATAC))
p2g$gene <- rowData(mRNA)$name
p2g$link_name<-paste(p2g$peak,'_',p2g$gene, sep = "")
head(p2g)

In [ ]:
# loading the peak2gene links from the fetal project 
p2g_invivo<-read.csv('cardiomyogenesis/p2g_invivo_allstates.csv')
head(p2g_invivo)

head(p2g_invivo[!p2g_invivo$link_name %in% c(p2g$link_name) ,])
head(p2g[p2g$link_name %in% c(as.character(p2g_invivo$link_name)) ,])

dim(p2g_invivo[!p2g_invivo$link_name %in% c(p2g$link_name) ,])
dim(p2g[p2g$link_name %in% c(as.character(p2g_invivo$link_name)) ,])


In [ ]:
#Getting the subset of peak 2 gene links that are only in fetal and that are shared between fetal and invitro
only_invivo<-p2g_invivo[!p2g_invivo$link_name %in% c(p2g$link_name) ,]
shared_invivo<-p2g_invivo[p2g_invivo$link_name %in% c(p2g$link_name) ,]
dim(only_invivo)
dim(shared_invivo)

In [ ]:
only_invivo_peak<-as.character(p2g_invivo[!p2g_invivo$link_name %in% c(p2g$link_name) ,'peak'])
head(only_invivo_peak)

In [ ]:
corCutOff = 0.45
FDRCutOff = 1e-04 
varCutOffATAC = 0.25 
varCutOffRNA = 0.25
k = 25
nPlot = 25000
filterp2g=NULL
limitsATAC = c(-2, 2) 
limitsRNA = c(-2, 2) 
groupBy = "Clusters1" 
palGroup=c("Z_Endocardium/Capillaries"="#FF8B74",
                                     "L_SMC"="#00CC00","J_Epicardium"="#60824F","M_Cardiac Fiborblast"="#00A08A","L_SMC"="#BED678",
                                     "H_Mature Cardiomyocytes"="#E22929","G_Primitive Cardiomyocytes"="#FCB31A")
palATAC = paletteContinuous("solarExtra")
palRNA = paletteContinuous("blueYellow")
verbose = TRUE 
returnMatrices = FALSE
seed = 1
ccd <- getCellColData(proj, select = groupBy)
pal <- paletteDiscrete(values = gtools::mixedsort(unique(ccd[, 1])))
pal[names(palGroup)[names(palGroup) %in% names(pal)]] <- palGroup[names(palGroup) %in% names(pal)]
#these are the lines that either plot the shared peak2 gene links or just the exclusive list either in fetal or invitro
mATAC <- readRDS(metadata(p2g)$seATAC)[only_invivo$idxATAC, ]
mRNA <- readRDS(metadata(p2g)$seRNA)[only_invivo$idxRNA, ]
mATAC <- assay(mATAC)
mRNA <- assay(mRNA)
dim(mATAC)
dim(mRNA)
KNNList <- as(metadata(readRDS(metadata(p2g)$seRNA))$KNNList, "list")
KNNGroups <- lapply(seq_along(KNNList), function(x) {
        KNNx <- KNNList[[x]]
        names(sort(table(ccd[KNNx, 1, drop = TRUE]), decreasing = TRUE))[1]}) %>% unlist
cD <- DataFrame(row.names = paste0("K", seq_len(ncol(mATAC))), groupBy = KNNGroups)
#pal <- paletteDiscrete(values = gtools::mixedsort(unique(ccd[, 1])))
colorMap <- list(groupBy = pal)
attr(colorMap[[1]], "discrete") <- TRUE
mATAC <- .rowZscores(mATAC)
mRNA <- .rowZscores(mRNA)
rownames(mATAC) <- NULL
rownames(mRNA) <- NULL
colnames(mATAC) <- paste0("K_", seq_len(ncol(mATAC)))
colnames(mRNA) <- paste0("K_", seq_len(ncol(mRNA)))
rownames(mATAC) <- paste0("P2G_", seq_len(nrow(mATAC)))
rownames(mRNA) <- paste0("P2G_", seq_len(nrow(mRNA)))
rownames(p2g) <- paste0("P2G_", seq_len(nrow(p2g)))
k1 <- kmeans(mATAC, k)
kDF <- DataFrame(k = k1$cluster, idx = seq_len(nrow(mATAC)))

bS <- .binarySort(t(.groupMeans(t(mATAC[kDF[, 2], ]), kDF[, 
        1])), clusterCols = TRUE, cutOff = 1)
    rowOrder <- rownames(bS[[1]])
    colOrder <- colnames(bS[[1]])
    kDF[, 3] <- as.integer(mapLabels(paste0(kDF[, 1]), newLabels = paste0(seq_along(rowOrder)), 
        oldLabels = rowOrder))
htATAC <- .ArchRHeatmap(mat = mATAC[kDF[, 2], gsub("^K","K_",rownames(cD[order(cD),,drop=FALSE])) ], 
        scale = FALSE, limits = limitsATAC, color = palATAC, 
        colData = cD[order(cD),,drop=FALSE], colorMap = colorMap, 
        clusterCols = FALSE, clusterRows = FALSE, split = kDF[, 
            3], labelRows = FALSE, labelCols = FALSE, draw = FALSE, 
        name = paste0("ATAC Z-Scores\n", nrow(mATAC), " P2GLinks"))

htRNA <- .ArchRHeatmap(mat = mRNA[kDF[, 2], gsub("^K","K_",rownames(cD[order(cD),,drop=FALSE])) ], scale = FALSE, 
        limits = limitsRNA, color = palRNA, colData = cD[order(cD),,drop=FALSE],
                           colorMap = colorMap, clusterCols = FALSE, 
        clusterRows = FALSE, split = kDF[, 3], labelRows = FALSE, 
        labelCols = FALSE, draw = FALSE, name = paste0("RNA Z-Scores\n", 
            nrow(mRNA), " P2GLinks"))

p2ghm<-htATAC+htRNA
#plotPDF(p2ghm, name = "PEAK2GENEHEATMAP_INVITRO_shared_invivo_invitro", width = 12, height = 12, ArchRProj = proj, addDOC = FALSE)
plotPDF(p2ghm, name = "PEAK2GENEHEATMAP_INVITRO_only_invivo", width = 12, height = 12, ArchRProj = proj, addDOC = FALSE)


In [ ]:
#similar procedure for the fetal project and instead 

In [ ]:
corCutOff = 0.45
FDRCutOff = 1e-04 
varCutOffATAC = 0.25 
varCutOffRNA = 0.25
k = 25
nPlot = 25000
filterp2g=NULL
limitsATAC = c(-2, 2) 
limitsRNA = c(-2, 2) 
groupBy = "Clusters2" 
palGroup=c(
"B_Atrial_Cardiomyocytes"="#FFAC53","C_Ventricular_Cardiomyocytes"="#F15F30","A_Myocardium"="#FFE500"
,"HA_Cardiac_Fibroblast"="#3B9AB2","F_Cardiac_fibroblast_progenitors"="#0AD7D3","K_vSMC"="#A8DDB5","L_Pericytes"="#79FFFF"
,"N_Neural_Crest"="#D0CD47","H_Early_Cardiac_fibroblast"="#90D5E4","D_Early_OFT_SMC"="#AAD962","M_Undifferentiated_epicardium"="#208A42"
,"E_valveFibroblast"="#74A9FF","G_valveFibroblast_late"="#74A9CF","J_Vasculatur_development"="#74C476","O_Endocaridum"="#C06CAB","P_Endocaridum_transitioning"="#C06CFF","T_Venal_endothelium"="#EC6BB1"
,"Q_Endocaridum_unidentifed"="#89288F","S_Capillary_endothelium"="#E6C2DC","R_Arterial_endothelium"="#F37B7D"
)
palATAC = paletteContinuous("solarExtra")
palRNA = paletteContinuous("blueYellow")
verbose = TRUE 
returnMatrices = FALSE
seed = 1
ccd <- getCellColData(proj, select = groupBy)
pal <- paletteDiscrete(values = gtools::mixedsort(unique(ccd[, 1])))
pal[names(palGroup)[names(palGroup) %in% names(pal)]] <- palGroup[names(palGroup) %in% names(pal)]
#these are the lines that either plot the shared peak2 gene links or just the exclusive list either in fetal or invitro
mATAC <- readRDS(metadata(p2g)$seATAC)[only_invivo$idxATAC, ]
mRNA <- readRDS(metadata(p2g)$seRNA)[only_invivo$idxRNA, ]
mATAC <- assay(mATAC)
mRNA <- assay(mRNA)
dim(mATAC)
dim(mRNA)
KNNList <- as(metadata(readRDS(metadata(p2g)$seRNA))$KNNList, "list")
KNNGroups <- lapply(seq_along(KNNList), function(x) {
        KNNx <- KNNList[[x]]
        names(sort(table(ccd[KNNx, 1, drop = TRUE]), decreasing = TRUE))[1]}) %>% unlist
cD <- DataFrame(row.names = paste0("K", seq_len(ncol(mATAC))), groupBy = KNNGroups)
#pal <- paletteDiscrete(values = gtools::mixedsort(unique(ccd[, 1])))
colorMap <- list(groupBy = pal)
attr(colorMap[[1]], "discrete") <- TRUE
mATAC <- .rowZscores(mATAC)
mRNA <- .rowZscores(mRNA)
rownames(mATAC) <- NULL
rownames(mRNA) <- NULL
colnames(mATAC) <- paste0("K_", seq_len(ncol(mATAC)))
colnames(mRNA) <- paste0("K_", seq_len(ncol(mRNA)))
rownames(mATAC) <- paste0("P2G_", seq_len(nrow(mATAC)))
rownames(mRNA) <- paste0("P2G_", seq_len(nrow(mRNA)))
rownames(p2g) <- paste0("P2G_", seq_len(nrow(p2g)))
k1 <- kmeans(mATAC, k)
kDF <- DataFrame(k = k1$cluster, idx = seq_len(nrow(mATAC)))

bS <- .binarySort(t(.groupMeans(t(mATAC[kDF[, 2], ]), kDF[, 
        1])), clusterCols = TRUE, cutOff = 1)
    rowOrder <- rownames(bS[[1]])
    colOrder <- colnames(bS[[1]])
    kDF[, 3] <- as.integer(mapLabels(paste0(kDF[, 1]), newLabels = paste0(seq_along(rowOrder)), 
        oldLabels = rowOrder))
htATAC <- .ArchRHeatmap(mat = mATAC[kDF[, 2], gsub("^K","K_",rownames(cD[order(cD),,drop=FALSE])) ], 
        scale = FALSE, limits = limitsATAC, color = palATAC, 
        colData = cD[order(cD),,drop=FALSE], colorMap = colorMap, 
        clusterCols = FALSE, clusterRows = FALSE, split = kDF[, 
            3], labelRows = FALSE, labelCols = FALSE, draw = FALSE, 
        name = paste0("ATAC Z-Scores\n", nrow(mATAC), " P2GLinks"))

htRNA <- .ArchRHeatmap(mat = mRNA[kDF[, 2], gsub("^K","K_",rownames(cD[order(cD),,drop=FALSE])) ], scale = FALSE, 
        limits = limitsRNA, color = palRNA, colData = cD[order(cD),,drop=FALSE],
                           colorMap = colorMap, clusterCols = FALSE, 
        clusterRows = FALSE, split = kDF[, 3], labelRows = FALSE, 
        labelCols = FALSE, draw = FALSE, name = paste0("RNA Z-Scores\n", 
            nrow(mRNA), " P2GLinks"))

p2ghm<-htATAC+htRNA
#plotPDF(p2ghm, name = "PEAK2GENEHEATMAP_FETAL_", width = 12, height = 12, ArchRProj = proj, addDOC = FALSE)
plotPDF(p2ghm, name = "PEAK2GENEHEATMAP_FETAL_only_invivo", width = 12, height = 12, ArchRProj = proj, addDOC = FALSE)
